<a href="https://colab.research.google.com/github/alexacheui/ateema_capstone/blob/main/%5BVersion_2%5D_Deterministic_classed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/My Drive/ADSP Capstone/Ateema')

In [4]:
!pwd

/content/drive/My Drive/ADSP Capstone/Ateema


In [8]:
import re
import os
import pandas as pd
from PIL import Image
import requests

class RTFProcessor:
    def __init__(self, file_path):
        self.file_path = file_path

    def process_rtf(self):
        # Read and clean RTF content
        with open(self.file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        content = re.sub(r'\\[a-zA-Z]+\d*', '', content)  # Remove commands
        content = re.sub(r'{|}', '', content)             # Remove braces
        content = re.sub(r'\\\*\\[^ ]+ ', '', content)    # Remove special characters
        content = re.sub(r'\n|\r', '', content)           # Remove newlines

        # Extract titles and URLs
        title_pattern = r'"Title": "(.*?)"'
        url_pattern = r'"URL": "(.*?)"'
        titles = re.findall(title_pattern, content)
        urls = re.findall(url_pattern, content)

        # Filter out entries with "No URL available"
        filtered_data = {"Title": [], "URL": []}
        for title, url in zip(titles, urls):
            if url != "No URL available":
                filtered_data["Title"].append(title)
                filtered_data["URL"].append(url)

        # Create a DataFrame
        df = pd.DataFrame(filtered_data)

        # Create 'video_image' folder if it doesn't exist
        if not os.path.exists('video_image'):
            os.makedirs('video_image')

        # Download and process images
        for index, row in df.iterrows():
            title = row['Title'].replace(' ', '_')
            url = row['URL']
            image_path = os.path.join('video_image', f"{title}.png")

            # Download the image
            response = requests.get(url)
            if response.status_code == 200:
                with open(image_path, 'wb') as file:
                    file.write(response.content)
                print(f"Downloaded: {image_path}")

                # Convert to PNG and save
                img = Image.open(image_path).convert("RGB")
                png_filename = os.path.splitext(os.path.basename(image_path))[0] + ".png"
                png_path = os.path.join('video_image', png_filename)
                img.save(png_path, "PNG")
                print(f"Converted and saved: {png_path}")
            else:
                print(f"Failed to download: {image_path}")

if __name__ == "__main__":
    # Path to the RTF file
    file_path = 'Final_Output.rtf'

    # Create an instance of the RTFProcessor
    processor = RTFProcessor(file_path)

    # Process the RTF file
    processor.process_rtf()


Downloaded: video_image/Alinea.png
Converted and saved: video_image/Alinea.png
Downloaded: video_image/City_Winery.png
Converted and saved: video_image/City_Winery.png
Downloaded: video_image/The_Art_Institute_of_Chicago.png
Converted and saved: video_image/The_Art_Institute_of_Chicago.png
Downloaded: video_image/Navy_Pier.png
Converted and saved: video_image/Navy_Pier.png
